## Import Library

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

## Data Pre-processing

In [2]:
#import data
df = pd.read_csv('dataset/ab_testing_dataset_100k.csv')
df.head()

,user_id,variant,date,device,browser,country,page_view,click,add_to_cart,purchase,revenue,session_duration,impressions,traffic_source,gender,age
0,1,A,2025-01-09,Mobile,Chrome,TH,10,0,0,0,100000,122,1,TikTok Ads,M,20
1,2,B,2025-01-04,Tablet,Edge,ID,6,1,0,0,50000,325,24,Organic,F,21
2,3,A,2025-01-06,Mobile,Firefox,MY,4,0,0,0,200000,446,25,Google Ads,F,38
3,4,A,2025-01-24,Tablet,Safari,TH,2,1,1,0,50000,441,11,Organic,F,42
4,5,A,2025-01-26,Desktop,Edge,ID,5,0,0,0,100000,188,26,TikTok Ads,F,40


In [3]:
#inspect data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           100000 non-null  int64 
 1   variant           100000 non-null  object
 2   date              100000 non-null  object
 3   device            100000 non-null  object
 4   browser           100000 non-null  object
 5   country           100000 non-null  object
 6   page_view         100000 non-null  int64 
 7   click             100000 non-null  int64 
 8   add_to_cart       100000 non-null  int64 
 9   purchase          100000 non-null  int64 
 10  revenue           100000 non-null  int64 
 11  session_duration  100000 non-null  int64 
 12  impressions       100000 non-null  int64 
 13  traffic_source    100000 non-null  object
 14  gender            100000 non-null  object
 15  age               100000 non-null  int64 
dtypes: int64(9), object(7)
memory usage: 12

In [4]:
#cek duplikasi data
df.duplicated().sum()

np.int64(0)

In [5]:
#cek missing value
df.isna().sum()

user_id             0
variant             0
date                0
device              0
browser             0
country             0
page_view           0
click               0
add_to_cart         0
purchase            0
revenue             0
session_duration    0
impressions         0
traffic_source      0
gender              0
age                 0
dtype: int64

In [6]:
#konversi tipe data yang belum sesuai
#konversi kolom survey date
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df.head()

,user_id,variant,date,device,browser,country,page_view,click,add_to_cart,purchase,revenue,session_duration,impressions,traffic_source,gender,age
0,1,A,2025-01-09,Mobile,Chrome,TH,10,0,0,0,100000,122,1,TikTok Ads,M,20
1,2,B,2025-01-04,Tablet,Edge,ID,6,1,0,0,50000,325,24,Organic,F,21
2,3,A,2025-01-06,Mobile,Firefox,MY,4,0,0,0,200000,446,25,Google Ads,F,38
3,4,A,2025-01-24,Tablet,Safari,TH,2,1,1,0,50000,441,11,Organic,F,42
4,5,A,2025-01-26,Desktop,Edge,ID,5,0,0,0,100000,188,26,TikTok Ads,F,40


In [7]:
#memisahkan data kategorikal dan numerik
nums = df.select_dtypes(include=['number']).columns.tolist()
cats = df.select_dtypes(exclude=['number']).columns.tolist()
df[nums].head()

,user_id,page_view,click,add_to_cart,purchase,revenue,session_duration,impressions,age
0,1,10,0,0,0,100000,122,1,20
1,2,6,1,0,0,50000,325,24,21
2,3,4,0,0,0,200000,446,25,38
3,4,2,1,1,0,50000,441,11,42
4,5,5,0,0,0,100000,188,26,40


In [8]:
df[cats].head()

,variant,date,device,browser,country,traffic_source,gender
0,A,2025-01-09,Mobile,Chrome,TH,TikTok Ads,M
1,B,2025-01-04,Tablet,Edge,ID,Organic,F
2,A,2025-01-06,Mobile,Firefox,MY,Google Ads,F
3,A,2025-01-24,Tablet,Safari,TH,Organic,F
4,A,2025-01-26,Desktop,Edge,ID,TikTok Ads,F


In [9]:
df.describe().round(2)

,user_id,date,page_view,click,add_to_cart,purchase,revenue,session_duration,impressions,age
count,100000.00,100000,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00
mean,50000.50,2025-01-15 12:14:34.367999744,7.48,0.30,0.15,0.07,124799.00,314.64,14.98,36.05
min,1.00,2025-01-01 00:00:00,1.00,0.00,0.00,0.00,0.00,30.00,1.00,18.00
25%,25000.75,2025-01-08 00:00:00,4.00,0.00,0.00,0.00,50000.00,172.00,8.00,27.00
50%,50000.50,2025-01-15 00:00:00,7.00,0.00,0.00,0.00,100000.00,315.00,15.00,36.00
75%,75000.25,2025-01-23 00:00:00,11.00,1.00,0.00,0.00,200000.00,457.00,22.00,45.00
max,100000.00,2025-01-30 00:00:00,14.00,1.00,1.00,1.00,250000.00,599.00,29.00,54.00
std,28867.66,NaN,4.03,0.46,0.36,0.26,85376.71,164.45,8.36,10.67


In [10]:
#mendeskripsikan data
df.describe(include='object')

,variant,device,browser,country,traffic_source,gender
count,100000,100000,100000,100000,100000,100000
unique,2,3,4,5,4,2
top,B,Desktop,Edge,TH,Organic,M
freq,50063,33607,25134,20223,25268,50025


In [11]:
#mencari outlier dengan metode IQR
Q1 = df[nums].quantile(0.25)
Q3 = df[nums].quantile(0.75)
IQR = Q3 -Q1

outliers = ((df[nums] < (Q1 - 1.5 * IQR)) | (df[nums] > (Q3 + 1.5 * IQR)))
print(outliers.sum())

user_id                 0
page_view               0
click                   0
add_to_cart         15064
purchase             7061
revenue                 0
session_duration        0
impressions             0
age                     0
dtype: int64


## Feature Engineering

1. Membuat kolom baru yaitu engagement category yang mengelompokkan responden berdasarkan session durationnya untuk mengetahui kebiasaan pengguna atau user behavior dalam 3 kategori yaitu low, moderate, dan high engagement

In [12]:
df['engagement_category'] = [
    'High Engagement' if n >= 480
    else 'Moderate Engagement' if n > 360 and n < 480
    else 'Low Engagement'
    for n in df['session_duration']
]
df[['user_id', 'variant', 'session_duration', 'engagement_category']].head()

,user_id,variant,session_duration,engagement_category
0,1,A,122,Low Engagement
1,2,B,325,Low Engagement
2,3,A,446,Moderate Engagement
3,4,A,441,Moderate Engagement
4,5,A,188,Low Engagement


groupby untuk menghitung jumlah user pada masing-masing kategori engagement

In [13]:
df.groupby('engagement_category')['user_id'].count().reset_index()

,engagement_category,user_id
0,High Engagement,21152
1,Low Engagement,58027
2,Moderate Engagement,20821


2. Mengkategorikan tingkat exposure pengguna berdasarkan kolom impression untuk memahami respon pengguna terhadap tingkat exposure

In [14]:
df['exposure_category'] = [
    'High Exposure' if n >= 25
    else 'Moderate Exposure' if n < 25 and n > 15
    else 'Low Exposure'
    for n in df['impressions']
]
df[['user_id', 'variant', 'impressions', 'exposure_category']].head()

,user_id,variant,impressions,exposure_category
0,1,A,1,Low Exposure
1,2,B,24,Moderate Exposure
2,3,A,25,High Exposure
3,4,A,11,Low Exposure
4,5,A,26,High Exposure


groupby untuk menghitung jumlah user pada masing-masing kategori exposure

In [15]:
df.groupby('exposure_category')['user_id'].count().reset_index()

,exposure_category,user_id
0,High Exposure,17140
1,Low Exposure,51919
2,Moderate Exposure,30941


3.mengkategorikan usia user untuk melihat  segmen mana yang paling responsif atau paling potensial secara bisnis

In [16]:
df['age_category'] = [
    'Eldery' if n >= 42
    else 'Adult' if n < 42 and n > 25
    else 'Young'
    for n in df['age']
]
df[['user_id', 'age', 'age_category']].head()

,user_id,age,age_category
0,1,20,Young
1,2,21,Young
2,3,38,Adult
3,4,42,Eldery
4,5,40,Adult


4. menghitung total session setiap user_id untuk nantinya digunakan dalam perhitungan conversion rate

In [17]:
total_session = df.groupby('user_id').size().reset_index(name='total_session')

df = df.merge(total_session, on='user_id', how='left')
df

,user_id,variant,date,device,browser,country,page_view,click,add_to_cart,purchase,revenue,session_duration,impressions,traffic_source,gender,age,engagement_category,exposure_category,age_category,total_session
0,1,A,2025-01-09,Mobile,Chrome,TH,10,0,0,0,100000,122,1,TikTok Ads,M,20,Low Engagement,Low Exposure,Young,1
1,2,B,2025-01-04,Tablet,Edge,ID,6,1,0,0,50000,325,24,Organic,F,21,Low Engagement,Moderate Exposure,Young,1
2,3,A,2025-01-06,Mobile,Firefox,MY,4,0,0,0,200000,446,25,Google Ads,F,38,Moderate Engagement,High Exposure,Adult,1
3,4,A,2025-01-24,Tablet,Safari,TH,2,1,1,0,50000,441,11,Organic,F,42,Moderate Engagement,Low Exposure,Eldery,1
4,5,A,2025-01-26,Desktop,Edge,ID,5,0,0,0,100000,188,26,TikTok Ads,F,40,Low Engagement,High Exposure,Adult,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,A,2025-01-11,Desktop,Chrome,ID,12,0,0,0,50000,177,26,TikTok Ads,F,47,Low Engagement,High Exposure,Eldery,1
99996,99997,B,2025-01-22,Desktop,Chrome,PH,5,0,0,1,100000,255,28,TikTok Ads,F,51,Low Engagement,High Exposure,Eldery,1
99997,99998,A,2025-01-20,Tablet,Firefox,SG,13,0,0,0,100000,205,28,Instagram,M,38,Low Engagement,High Exposure,Adult,1
99998,99999,B,2025-01-02,Mobile,Chrome,TH,10,0,1,0,150000,559,18,Instagram,M,47,High Engagement,Moderate Exposure,Eldery,1


4. melakukan sortir untuk mengurutkan variant A dan B, agar dataset lebih rapi 

In [18]:
df_sorted = df.sort_values(by=['variant', 'user_id'], ascending=True)
df_sorted

,user_id,variant,date,device,browser,country,page_view,click,add_to_cart,purchase,revenue,session_duration,impressions,traffic_source,gender,age,engagement_category,exposure_category,age_category,total_session
0,1,A,2025-01-09,Mobile,Chrome,TH,10,0,0,0,100000,122,1,TikTok Ads,M,20,Low Engagement,Low Exposure,Young,1
2,3,A,2025-01-06,Mobile,Firefox,MY,4,0,0,0,200000,446,25,Google Ads,F,38,Moderate Engagement,High Exposure,Adult,1
3,4,A,2025-01-24,Tablet,Safari,TH,2,1,1,0,50000,441,11,Organic,F,42,Moderate Engagement,Low Exposure,Eldery,1
4,5,A,2025-01-26,Desktop,Edge,ID,5,0,0,0,100000,188,26,TikTok Ads,F,40,Low Engagement,High Exposure,Adult,1
6,7,A,2025-01-01,Tablet,Edge,MY,12,0,1,0,200000,342,12,Instagram,M,18,Low Engagement,Low Exposure,Young,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99992,99993,B,2025-01-20,Desktop,Edge,MY,2,0,0,0,100000,37,8,Instagram,F,32,Low Engagement,Low Exposure,Adult,1
99994,99995,B,2025-01-06,Mobile,Firefox,MY,9,0,0,0,50000,377,3,Organic,M,32,Moderate Engagement,Low Exposure,Adult,1
99996,99997,B,2025-01-22,Desktop,Chrome,PH,5,0,0,1,100000,255,28,TikTok Ads,F,51,Low Engagement,High Exposure,Eldery,1
99998,99999,B,2025-01-02,Mobile,Chrome,TH,10,0,1,0,150000,559,18,Instagram,M,47,High Engagement,Moderate Exposure,Eldery,1


5. menata urutan kolom

In [19]:
cols = list(df.columns)

# Pindahkan 'gender' dan 'age' ke setelah 'date'
cols.insert(cols.index('date') + 1, cols.pop(cols.index('gender')))
cols.insert(cols.index('date') + 2, cols.pop(cols.index('age')))
cols.insert(cols.index('age') + 1, cols.pop(cols.index('age_category')))
cols.insert(cols.index('session_duration') + 1, cols.pop(cols.index('engagement_category')))
cols.insert(cols.index('impressions') + 1, cols.pop(cols.index('exposure_category')))

df = df[cols]
df

,user_id,variant,date,gender,age,age_category,device,browser,country,page_view,click,add_to_cart,purchase,revenue,session_duration,engagement_category,impressions,exposure_category,traffic_source,total_session
0,1,A,2025-01-09,M,20,Young,Mobile,Chrome,TH,10,0,0,0,100000,122,Low Engagement,1,Low Exposure,TikTok Ads,1
1,2,B,2025-01-04,F,21,Young,Tablet,Edge,ID,6,1,0,0,50000,325,Low Engagement,24,Moderate Exposure,Organic,1
2,3,A,2025-01-06,F,38,Adult,Mobile,Firefox,MY,4,0,0,0,200000,446,Moderate Engagement,25,High Exposure,Google Ads,1
3,4,A,2025-01-24,F,42,Eldery,Tablet,Safari,TH,2,1,1,0,50000,441,Moderate Engagement,11,Low Exposure,Organic,1
4,5,A,2025-01-26,F,40,Adult,Desktop,Edge,ID,5,0,0,0,100000,188,Low Engagement,26,High Exposure,TikTok Ads,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,A,2025-01-11,F,47,Eldery,Desktop,Chrome,ID,12,0,0,0,50000,177,Low Engagement,26,High Exposure,TikTok Ads,1
99996,99997,B,2025-01-22,F,51,Eldery,Desktop,Chrome,PH,5,0,0,1,100000,255,Low Engagement,28,High Exposure,TikTok Ads,1
99997,99998,A,2025-01-20,M,38,Adult,Tablet,Firefox,SG,13,0,0,0,100000,205,Low Engagement,28,High Exposure,Instagram,1
99998,99999,B,2025-01-02,M,47,Eldery,Mobile,Chrome,TH,10,0,1,0,150000,559,High Engagement,18,Moderate Exposure,Instagram,1


In [20]:
df_clean = df.copy()
df_clean.to_csv('data_abtesting_project.csv', index=False)